In [2]:
import en_core_web_sm
nlp = en_core_web_sm.load()
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import pandas as pd
import numpy as np
from time import time 
import multiprocessing
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split

#file_name = 'document_classification\Resume1.txt'
#mat=[][2]
import os


main=[]
#print(files)
for i in os.listdir("/content/drive/My Drive/resume/Training & Test/Resume"):
     if i != ".ipynb_checkpoints":
      x=i
      try:
        fname = open("/content/drive/My Drive/resume/Training & Test/Resume/"+x).read()
      except UnicodeDecodeError:
         continue
        
      f1=re.sub(r'\W+', ' ',fname)
    
      wordnet_lemmatizer = WordNetLemmatizer()
    
      words = nltk.word_tokenize(f1)
    
      f2=" "
      for w in words:
          f2=f2+" "+wordnet_lemmatizer.lemmatize(w)
    
      doc = nlp(f2)
    
      tokens = [token.text for token in doc if not token.is_stop]
    
      l=[]
      l.append(tokens)
      l.append("1")
      main.append(l)

for i in os.listdir("/content/drive/My Drive/resume/Training & Test/Non Resume"):
     if i != ".ipynb_checkpoints":
       x=i
       try:
          fname = open("/content/drive/My Drive/resume/Training & Test/Non Resume/"+x).read()
       except UnicodeDecodeError:
          continue
        
       f1=re.sub(r'\W+', ' ',fname)
       wordnet_lemmatizer = WordNetLemmatizer()
       words = nltk.word_tokenize(f1)
       f2=" "
       for w in words:
           f2=f2+" "+wordnet_lemmatizer.lemmatize(w)
       doc = nlp(f2)
    
       tokens = [token.text for token in doc if not token.is_stop]
      
       l=[]
       l.append(tokens)
       l.append("0")
       main.append(l)



df = pd.DataFrame(main, columns = ['Tokens', 'Label'])







LookupError: ignored

In [0]:

pred = []
for i in os.listdir("/content/drive/My Drive/resume/Prediction/Resume"):
    if i != ".ipynb_checkpoints":
      x=i
      try:
        fname = open("/content/drive/My Drive/resume/Prediction/Resume/"+x).read()
      except UnicodeDecodeError:
        continue
        
      f1=re.sub(r'\W+', ' ',fname)
      wordnet_lemmatizer = WordNetLemmatizer()
      words = nltk.word_tokenize(f1)
      f2=" "
      for w in words:
          f2=f2+" "+wordnet_lemmatizer.lemmatize(w)
      doc = nlp(f2)
   
      tokens = [token.text for token in doc if not token.is_stop]
    
      l=[]
      l.append(tokens)
      pred.append(l)

print(len(pred))

for i in os.listdir("/content/drive/My Drive/resume/Prediction/Non Resume"):
    if i != ".ipynb_checkpoints":
      x=i
      try:
          fname = open("/content/drive/My Drive/resume/Prediction/Non Resume/"+x).read()
      except UnicodeDecodeError:
          continue
          
      f1=re.sub(r'\W+', ' ',fname)
      wordnet_lemmatizer = WordNetLemmatizer()
      words = nltk.word_tokenize(f1)
      f2=" "
      for w in words:
          f2=f2+" "+wordnet_lemmatizer.lemmatize(w)
      doc = nlp(f2)
    
      tokens = [token.text for token in doc if not token.is_stop]
      
      l=[]
      l.append(tokens)
      pred.append(l)

print(len(pred))

df_pred = pd.DataFrame(pred, columns=['Tokens'])
df_pred

4
8


,Tokens
0,"[ , Computer, Science, Faculty, New, Delhi, D..."
1,"[ , Isabelle, Smith, rn, cphon, Sometown, TX,..."
2,"[ , Don, t, know, begin, Click, link, resume,..."
3,"[ , Don, t, know, begin, Click, link, resume,..."
4,"[ , BIODATA, G, Pranathi, 8143958515, 16wh1a0..."
5,"[ , PAYMENT, RECEIPT, Guest, Sanjay, Date, 31..."
6,"[ , Jax, Sampson, 111, 789, 3456, jax, sampso..."
7,"[ , Cody, Fredrickson, 123, 456, 7891, cfredr..."


In [0]:
cores = multiprocessing.cpu_count() 
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=1000,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

t = time()

print(w2v_model.corpus_count)
w2v_model.build_vocab(df["Tokens"], progress_per=1000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

w2v_model.train(df["Tokens"], total_examples=w2v_model.corpus_count, epochs=10000, report_delay=1)
print(w2v_model.corpus_count)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

0
Time to build vocab: 0.0 mins
194
Time to train the model: 16.09 mins


In [0]:
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix

y = df['Label'].values
X = np.array(df["Tokens"])

#And here is the train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)


vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)

matrix = vectorizer.fit_transform([x for x in X_train])

tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    
    count = 0.
    for word in tokens:
        try:
    
            vec += w2v_model[word].reshape((1, size)) * tfidf[word]
            print(w2v_model[word])
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    
    
    return vec

train_vecs_w2v = np.concatenate([buildWordVector(z, 1000) for z in map(lambda x: x, X_train)])


test_vecs_w2v = np.concatenate([buildWordVector(z, 1000) for z in map(lambda x: x, X_test)])
train_vecs_w2v = scale(train_vecs_w2v)


test_vecs_w2v = scale(test_vecs_w2v)

print(test_vecs_w2v)


vocab size : 639


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:


X_pred = np.array(df_pred["Tokens"])


pred_vec_w2v = np.concatenate([buildWordVector(z, 1000) for z in map(lambda x: x, X_pred)])
pred_vec_w2v = scale(pred_vec_w2v)

print(pred_vec_w2v)

print ('shape for training set : ',train_vecs_w2v.shape,
      '\nshape for test set : ', test_vecs_w2v.shape,
       '\nshape for predicted set : ', pred_vec_w2v.shape,) 


[[-0.99728757  0.22012779  0.28329361 ...  2.31811969  1.29675151
   1.93578235]
 [-1.08099882 -2.12974629  1.80628943 ... -1.12633679 -2.04906944
   0.39463985]
 [ 1.11533768  0.97517249 -1.37097711 ... -0.69088525  0.30290978
  -0.61131167]
 ...
 [-0.7938815  -0.86028872  0.72150687 ...  0.62846081  0.55708205
   0.80772044]
 [ 0.34164224 -0.18166186 -0.79304318 ...  0.02268002 -0.41036219
  -0.34792282]
 [ 1.73955098  1.06980832 -0.96129792 ... -0.54350949 -0.25826497
  -1.58018604]]
shape for training set :  (135, 300) 
shape for test set :  (59, 300) 
shape for predicted set :  (8, 300)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [0]:
import keras 
from keras.models import Sequential, Model 
from keras import layers
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Input, Embedding
from keras.layers.merge import Concatenate

model1 = Sequential()
 
model1.add(Dense(128, activation='relu', input_dim=1000))
model1.add(Dropout(0.7))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               128128    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 128,257
Trainable params: 128,257
Non-trainable params: 0
_________________________________________________________________


In [0]:
import matplotlib.pyplot as plt

history = model1.fit(train_vecs_w2v, y_train, epochs=40, batch_size=50,
                   validation_data=(test_vecs_w2v,y_test))

loss, accuracy = model1.evaluate(train_vecs_w2v, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))


loss, accuracy = model1.evaluate(test_vecs_w2v, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

model1.save("/content/sample_data/myfirstmodel", overwrite=True, include_optimizer=True)



Train on 135 samples, validate on 59 samples
Epoch 1/40
135/135 [==============================] - 0s 2ms/step - loss: 0.6930 - acc: 0.5556 - val_loss: 0.6929 - val_acc: 0.5424
Epoch 2/40
135/135 [==============================] - 0s 186us/step - loss: 0.6924 - acc: 0.6148 - val_loss: 0.6928 - val_acc: 0.5424
Epoch 3/40
135/135 [==============================] - 0s 173us/step - loss: 0.6919 - acc: 0.6148 - val_loss: 0.6926 - val_acc: 0.5424
Epoch 4/40
135/135 [==============================] - 0s 155us/step - loss: 0.6915 - acc: 0.6148 - val_loss: 0.6924 - val_acc: 0.5424
Epoch 5/40
135/135 [==============================] - 0s 159us/step - loss: 0.6911 - acc: 0.6148 - val_loss: 0.6923 - val_acc: 0.5424
Epoch 6/40
135/135 [==============================] - 0s 171us/step - loss: 0.6906 - acc: 0.6148 - val_loss: 0.6921 - val_acc: 0.5424
Epoch 7/40
135/135 [==============================] - 0s 144us/step - loss: 0.6902 - acc: 0.6148 - val_loss: 0.6920 - val_acc: 0.5424
Epoch 8/40
135/135 

In [1]:

l = model1.predict(pred_vec_w2v, verbose=True)
print(l)

NameError: ignored

In [0]:
~ ls

NameError: ignored